In [1]:
import tensorflow as tf
from keras.datasets import cifar10
from keras.models import Model
from keras.layers import Dense, Activation, Flatten, Input, Add
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.optimizers import Adam
from keras.utils import to_categorical

In [2]:
# Load the CIFAR-10 data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Preprocess data
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

170498071/170498071 [==============================] - 26s 0us/step


In [3]:

# Define a function to create a residual block
def residual_block(x, filters):
    res = Conv2D(filters, (3, 3), padding='same')(x)
    res = BatchNormalization()(res)
    res = Activation('relu')(res)
    res = Conv2D(filters, (3, 3), padding='same')(res)
    res = BatchNormalization()(res)

    shortcut = Conv2D(filters, (1, 1), padding='same')(x)
    shortcut = BatchNormalization()(shortcut)

    res = Add()([shortcut, res])
    return Activation('relu')(res)

# Input tensor
inputs = Input(shape=(32, 32, 3))

# Create a residual network
x = residual_block(inputs, 32)
x = MaxPooling2D((2, 2))(x)
x = residual_block(x, 64)
x = MaxPooling2D((2, 2))(x)
x = residual_block(x, 128)
x = Flatten()(x)
outputs = Dense(10, activation='softmax')(x)

# Create the model
model = Model(inputs, outputs)

# Compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [4]:

# Train the model
model.fit(x_train, y_train, batch_size=64, epochs=1, validation_data=(x_test, y_test))


2023-06-30 19:08:44.466551: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


782/782 [==============================] - 84s 107ms/step - loss: 1.2510 - accuracy: 0.5712 - val_loss: 1.2606 - val_accuracy: 0.5889
